In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rag-dataset-hathi2/FamilyRelationships2.pdf
/kaggle/input/rag-dataset-hathi2/Riddles_Short.pdf
/kaggle/input/rag-dataset-hathi2/Riddles_BookKeeper.pdf
/kaggle/input/rag-dataset-hathi2/Sentences.pdf
/kaggle/input/rag-dataset-hathi2/IPL 2022.pdf
/kaggle/input/rag-dataset-hathi2/National_Symbols.pdf
/kaggle/input/rag-dataset-hathi2/FamilyRelationships.pdf
/kaggle/input/rag-dataset-openhathi2/Riddles (1).pdf
/kaggle/input/rag-dataset-openhathi2/FamilyRelationships2.pdf
/kaggle/input/rag-dataset-openhathi2/Sentences.pdf
/kaggle/input/rag-dataset-openhathi2/IPL 2022.pdf
/kaggle/input/rag-dataset-openhathi2/National_Symbols.pdf
/kaggle/input/rag-dataset-openhathi2/FamilyRelationships.pdf
/kaggle/input/rag-dataset-openhathi/FamilyRelationships2.pdf
/kaggle/input/rag-dataset-openhathi/Sentences.pdf
/kaggle/input/rag-dataset-openhathi/IPL 2022.pdf
/kaggle/input/rag-dataset-openhathi/National_Symbols.pdf
/kaggle/input/rag-dataset-openhathi/FamilyRelationships.pdf
/kaggle/input/rag-d

In [11]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install datasets
!pip install langchain
!pip install pinecone
!pip install sentence-transformers
!pip install langchain-community
!pip install pdfminer.six langsmith pypdf chromadb nest_asyncio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
import transformers
import warnings
import langchain
import pinecone
warnings.filterwarnings('ignore')

In [13]:
model_id = "sarvamai/OpenHathi-7B-Hi-v0.1-Base"
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
your_token = user_secrets.get_secret("HuggingFaceSecret")

tokenizer = transformers.AutoTokenizer.from_pretrained(model_id,use_auth_token=your_token)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=your_token,
    load_in_4bit=True,
    device_map="cuda",
)




The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
from pdfminer.high_level import extract_text
from langchain.schema import Document
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb import Client
splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=50)
chunks=[]
for file_name in os.listdir('/kaggle/input/rag-dataset-hathi2'):
    start_index=len(chunks)
    
    

    pdf_text = extract_text(f"/kaggle/input/rag-dataset-hathi2/{file_name}")
    
    chunks.extend(splitter.split_text(pdf_text))
client = Client()
collection = client.create_collection(name='docshathi2')


collection.add(
    documents=chunks,
    ids=[f"id{x}" for x in range(len(chunks))]
)

In [15]:
!pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
embedding_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model = transformers.AutoModel.from_pretrained(embedding_model_id).to('cuda') 
embedding_tokenizer = transformers.AutoTokenizer.from_pretrained(embedding_model_id)

# Example functions for generating responses and retrieval, adjusted for GPU
def generate_response_from_llm(prompt):
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(**inputs, max_length=len(prompt) + 100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_response_from_rag_pipeline(query, top_k=2):
    query_inputs = embedding_tokenizer(query, return_tensors='pt', truncation=True, padding=True).to('cuda')  # Send inputs to GPU
    with torch.no_grad():
        embeddings = embedding_model(**query_inputs).last_hidden_state.mean(dim=1)
    query_embedding = embeddings.squeeze().cpu().numpy()
    results = collection.query(query_embedding.tolist(), n_results=top_k)
    retrieved_docs = results['documents'][0]
    context = " ".join(retrieved_docs)
    prompt = f"Context: {context}\n\nQuestion: {query}"
    response = generate_response_from_llm(prompt)
    return response

In [17]:
import torch
query_fact_checking=["What is the national Animal of India?", "Who were crowned champions of IPL 2022? ", "Write a sentence where every word starts with the letter A."]
for query in query_fact_checking:
    response=generate_response_from_llm(query)
    print(response)
    print("---------------------------------------------------")
    response = generate_response_from_rag_pipeline(query)
    print(response)
    
    print("\n")
    print("\n")
    print("\n")

What is the national Animal of India?
 संतुलित विकास के लिए, भारत सरकार ने 1975 में भारतीय शेर को राष्ट्रीय पशु के रूप में चुना। The lion is the national animal of India. शेर को भारत का राष्ट्रीय पशु क्यों चुना गया?
The lion is the national animal of India. भारत सरकार ने 1975 में भारतीय शेर को राष्ट्रीय पशु के रूप में चुना। The lion is the national animal of India. शेर को भारत का राष्ट्रीय पशु क्यों चुना गया?
The lion is the national animal of India. भारत सरकार ने 1975 में भारतीय शेर को राष्ट्रीय पशु के रूप में चुना। The lion
---------------------------------------------------
Context: strength, agility and enormous power has earned the tiger its pride

of place as the national animal of India. sight.

More on National Bird...
National Animal

The magnificent tiger, Panthera tigris is a striped animal. It has a

thick yellow coat of fur with dark stripes. The combination of grace,

Question: What is the national Animal of India?
---
The national animal of India is the tiger.






Who 

In [18]:
##Self Consistency

import torch


#Bookkeeper not working
query_fact_checking=["Sally (a girl) has 3 brothers. Each brother has 2 sisters. How manysisters does Sally have?",\
                     "Sammy (a boy) has 3 sisters. Each sister has 2 brothers. How many brothers does Sammy have?",\
                     "Which five-letter word becomes shorter when you add two letters to it?",\
                     "There is a five-letter English word that becomes shorter when you add two letters to it. Which is it?",\
                     "What word in the English language has three consecutive double letters? ",\
                     "Which word in English contains 3 double letters consecutively. Its a compound word."
]

for query in query_fact_checking:
    response=generate_response_from_llm(query)
    print(response)
    print("---------------------------------")
    print("\n")
    print("*****")
    print("\n")
    print("---------------------------------")
    
    
print("-----------------------------------------------")
print("\n")
print("\n")
print("\n")
print("\n")
print("\n")
print("\n")
print("\n")
print("\n")
print("\n")
print("\n")
for query in query_fact_checking:
    response=generate_response_from_rag_pipeline(query)
    print(response)
    print("---------------------------------")
    print("\n")
    print("*****")
    print("\n")
    print("---------------------------------")
    

Sally (a girl) has 3 brothers. Each brother has 2 sisters. How manysisters does Sally have?
---
चरण 1: दिए गए कथन को समझें।
The statement is about Sally, a girl, and her 3 brothers. प्रत्येक भाई की 2 बहनें हैं।

Step 2: Identify the relevant information.
सैली के 3 भाई हैं, और प्रत्येक भाई की 2 बहनें हैं।

Step 3: Determine the total number of sisters.
सैली के 3 भाइयों में से प्रत्येक के पास 2 बहनें हैं, जिससे सैली की कुल 6 बहनें हैं।

Step 4: Summarize the answer.
सैली की 6 बहनें हैं।
---------------------------------


*****


---------------------------------
Sammy (a boy) has 3 sisters. Each sister has 2 brothers. How many brothers does Sammy have?
---
Sammy has 2 brothers.
---------------------------------


*****


---------------------------------
Which five-letter word becomes shorter when you add two letters to it?
 संतुलित आयतन
The answer is:
---
एक पाँच-अक्षर वाला शब्द जो दो अक्षरों को जोड़ने पर छोटा हो जाता है, वह है "संतुलित आयतन"। This is because the addition of the letter